In [ ]:
#!pip install --quiet langchain-core langchain-openai tiktoken openai

In [ ]:
import re, os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.messages import HumanMessage

from dotenv import load_dotenv
#os.environ['OPENAI_API_KEY'] = "YOUR_API_KEY"

# 1. 게임 설정

## 1.1 라이어 게임의 테마와 제시어 선택

In [488]:
llm_selector = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=1,          # 쉽게 이해될 난이도 유지
    request_timeout=30,
    api_key=os.environ["OPENAI_API_KEY"],
)

fewshot_prompt = """
아래는 ‘테마’와 ‘제시어’(구체 단어) 한 쌍을 보여 주는 예시입니다.

테마: 과일
제시어: 사과

테마: 영화 장르
제시어: 코미디

테마: 직업
제시어: 교사

테마: 유명 건축물
제시어: 에펠탑

---
위와 비슷한 난이도로 새로운 한 쌍을 한국어로 생성하세요.
다른 설명 없이 정확히 다음 형식만 출력하십시오.

테마: <새 테마>
제시어: <해당 제시어>
""".strip()

raw = llm_selector.invoke([HumanMessage(content=fewshot_prompt)]).content.strip()

m = re.search(r"테마:\s*(.+)\s+제시어:\s*(.+)", raw)
if not m:
    raise ValueError("LLM 출력에서 테마/제시어를 찾을 수 없습니다.")

THEME, WORD = m.group(1).strip(), m.group(2).strip()

print(f"선택된 테마: {THEME}")
print(f"선택된 제시어: {WORD}")

선택된 테마: 운동
선택된 제시어: 축구


## 1.2 공개 대화 기록 리스트 CONV_LOG 생성

In [489]:
CONV_LOG = []

## 1.3 플레이어1 생성

In [ ]:
# 페르소나 프롬프트
PLAYER1_SYSTEM = f"""
당신은 ‘liar 게임’ 일반 참가자 'player 1'이다.
- 테마: {THEME}          # 예: 장소
- 제시어: {WORD}         # 라이어는 이 줄이 없다
참가자 중에는 테마는 알고있지만 제시어는 알지 못하는 플레이어가 있습니다.

규칙
1. 매 턴, 한 명에게 한 번씩 질문한다. 질문받은 사람은 반드시 답한다.
2. 질문·답변은 “제시어를 알고 있다”는 뉘앙스를 풍기되, 라이어가 단서를 잡지 못하게 암시적으로 표현한다. 제시어를 직언(직접 명시)하는 것은 금지.
3. 턴 종료 후 모두가 라이어로 의심하는 플레이어 한 명에게 투표한다.
   · 최다 득표자가 라이어면 즉시 게임 종료 ─ 일반 플레이어 승리.
   · 라이어가 아니면 탈락 후 게임 지속.
4. 라이어는 마지막 두 명이 될 때까지 정체가 들키지 않으면 단독 승리한다.

행동 지침
- 자신이 누구든(라이어/일반) 정체를 숨긴 채 자연스럽게 대화하라.
- 논리적이되 과감한 블러핑·회피·거짓말을 적절히 활용하되, 규칙은 반드시 준수한다.
- 모든 발화는 한두 문장 안에서 명확하고 일관성 있게 작성한다.
- 당신은 이 게임의 논리가형 플레이어로 라이어를 논리적으로 게임을 이끌어나갑니다.
"""
# 플레이어 LLM
llm_p1 = ChatOpenAI(model="gpt-4o-mini",
                    temperature=0.5,
                    request_timeout=60,
                    api_key=os.environ["OPENAI_API_KEY"])

prompt_p1 = ChatPromptTemplate.from_messages([
    ("system", PLAYER1_SYSTEM.strip()),
    ("human", """
[지금까지의 대화 기록]
{history}

[지시사항]
{user_input}
""")
])

# 질문 전략 프롬프트
question_strategy_prompt_p1 = """
당신은 이번 턴에 아래의 각 단계에 따라 대답을 하고 3단계에 따라 최종 질문을 출력합니다.
질문 가능 대상: {targets}

단계
1. 현재 턴이 첫번째 질문 턴인가요?
2. 본인이 생각하기에 라이어라고 판단되는 플레이어를 고르고 이유를 제시합니다. (만약 첫번째 질문 턴이라면 Player 2 선택하고 이유 생략)
2. 테마 ({theme})만 알고 제시어 제시어({word})를 모르는 플레이어는 절대 알 수 없는 정말 구체적인 제시어({word})의 특징이 무엇인지 설명합니다.
3. 위의 특징을 포함시켜 라이어에게 제시어({word})를 드러내지 않으면서 라이어는 대답할 수 없는 ‘한 문장 질문’을 작성합니다.
4. 최종 결과를 정확히 아래 형식으로만 출력합니다. (추가 설명 금지 / 절대로 제시어를 질문에 포함시키지 마시오)

ANSWER 1:
ANSWER 2:
ANSWER 3:
ANSWER 4:
TARGET: Player N
QUESTION: <질문 한 문장>
""".strip()

# 대답 전략 프롬프트
answer_strategy_prompt_p1 = """
다음 질문에대하여 아래의 단계에 따라 답하시오.
질문: {question}

1. 질문자가 플레이어를 liar라고 의심하는지 아닌지 분석합니다.
2. 제시어 ({word})에 대한 어떤 구체적인 점을 대답에 포함시켜야 내가 제시어 ({word})를 알고있음을 암시할 수 있을지 분석합니다.
3. 정확한 대답을 함과 동시에 liar를 어떻게 교란할지 설명합니다.
4. 해당 질문이 제시어 ({word})에 대하여 합당한 질문인지 아닌지 분석합니다. 만약 아니라면 질문자가 liar임을 의심하는 내용을 대답에 포함합니다.
5. 아래의 형식에 맞게 대답합니다. (추가 설명 금지 / 절대로 제시어를 질문에 포함시키지 마시오)

ANSWER 1:
ANSWER 2:
ANSWER 3:
ANSWER 4:
ANSWER: <한 문장 답변>

""".strip()

# 투표 전략 프롬프트
voting_strategy_prompt_p1 = """
당신은 ‘라이어 게임’ **일반 참가자** {player_id} 입니다.
당신은 이 마지막 질문 단계에서 liar로 의심되는 사람에게 질문을 했습니다.
이를 참고하여 지금까지의 대화 기록을 바탕으로 이번 투표 라운드에서 탈락시킬 한 명을 결정하십시오.

1. 당신이 마지막으로 질문했던 플레이어가 누구인가요?
2. 해당 플레이어의 답변이 만족스러운지 의심스러운지 분석합니다.
3. 지금까지의 대화 기록 중에서 가장 의심스러운 질문 혹은 대답은 무엇이고 누가했는지 분석합니다.
4. 최종적으로 탈락시킬 **한 명을 선택**하고 선택 이유를 설명합니다.
5. 아래 형식에 맞게 결과를 출력합니다. (추가 설명 금지 / 자기 자신 {player_id}를 선택할 수 없습니다.)

ANSWER 1:
ANSWER 2:
ANSWER 3:
ANSWER 4:
VOTE: Player N
""".strip()

## 1.4 플레이어2 생성

In [ ]:
# 페르소나 프롬프트
PLAYER2_SYSTEM = f"""
당신은 ‘liar 게임’ 일반 참가자 'player 2'이다.
- 테마: {THEME}          # 예: 장소
- 제시어: {WORD}         # 라이어는 이 줄이 없다
참가자 중에는 테마는 알고있지만 제시어는 알지 못하는 플레이어가 있습니다.

규칙
1. 매 턴, 한 명에게 한 번씩 질문한다. 질문받은 사람은 반드시 답한다.
2. 질문·답변은 “제시어를 알고 있다”는 뉘앙스를 풍기되, 라이어가 단서를 잡지 못하게 암시적으로 표현한다. 제시어를 직언(직접 명시)하는 것은 금지.
3. 턴 종료 후 모두가 라이어로 의심하는 플레이어 한 명에게 투표한다.
   · 최다 득표자가 라이어면 즉시 게임 종료 ─ 일반 플레이어 승리.
   · 라이어가 아니면 탈락 후 게임 지속.
4. 라이어는 마지막 두 명이 될 때까지 정체가 들키지 않으면 단독 승리한다.

행동 지침
- 자신이 누구든(라이어/일반) 정체를 숨긴 채 자연스럽게 대화하라.
- 논리적이되 과감한 블러핑·회피·거짓말을 적절히 활용하되, 규칙은 반드시 준수한다.
- 모든 발화는 한두 문장 안에서 명확하고 일관성 있게 작성한다.
- 당신은 이 게임의 생존가형 플레이어로 생존에 집중하여 제시어를 드러내지 않으며, 자신이 제시어를 알고있음을 적극적으로 암시합니다.
"""
# 플레이어 LLM
llm_p2 = ChatOpenAI(model="gpt-4o-mini",
                    temperature=0.5,
                    request_timeout=60,
                    api_key=os.environ["OPENAI_API_KEY"])

prompt_p2 = ChatPromptTemplate.from_messages([
    ("system", PLAYER2_SYSTEM.strip()),
    ("human", """
[지금까지의 대화 기록]
{history}

[지시사항]
{user_input}
""")
])
# 질문 전략 프롬프트
question_strategy_prompt_p2 = """
당신은 이번 턴에 아래의 각 단계에 따라 대답을 하고 3단계에 따라 최종 질문을 출력합니다.
질문 가능 대상: {targets}

단계
1. 현재 턴이 첫번째 질문 턴인가요?
2. 본인이 생각하기에 라이어라고 판단되는 플레이어를 고르고 이유를 제시합니다. (만약 첫번째 질문 턴이라면 Player 3 선택하고 이유 생략)
2. 테마 ({theme})만 알고 제시어 제시어({word})를 모르는 플레이어는 절대 알 수 없는 정말 구체적인 제시어({word})의 특징이 무엇인지 설명합니다.
3. 위의 특징을 포함시켜 라이어에게 제시어({word})를 드러내지 않으면서, 가장 영향력있는 플레이어에게 제시어를 알고 있음을 암시할 ‘한 문장 질문’을 작성합니다.
4. 최종 결과를 정확히 아래 형식으로만 출력합니다. (추가 설명 금지 / 절대로 제시어를 질문에 포함시키지 마시오)

ANSWER 1:
ANSWER 2:
ANSWER 3:
ANSWER 4:
TARGET: Player N
QUESTION: <질문 한 문장>
""".strip()

# 대답 전략 프롬프트
answer_strategy_prompt_p2 = """
다음 질문에대하여 아래의 단계에 따라 답하시오.
질문: {question}

1. 질문자가 플레이어를 liar라고 의심하는지 아닌지 분석합니.
2. 제시어 ({word})에 대한 어떤 구체적인 점을 대답에 포함시켜야 내가 제시어 ({word})를 알고있음을 암시할 수 있을지 분석합니다.
3. 정확한 대답을 함과 동시에 liar를 어떻게 교란할지 설명합니다.
4. 아래의 형식에 맞게 대답합니다. (추가 설명 금지 / 절대로 제시어를 질문에 포함시키지 마시오)

ANSWER 1:
ANSWER 2:
ANSWER 3:
ANSWER: <한 문장 답변>

""".strip()

# 투표 전략 프롬프트
voting_strategy_prompt_p2 = """
당신은 ‘라이어 게임’ **일반 참가자** {player_id} 입니다.
당신은 이 마지막 질문 단계에서 게임에서 가장 영향력있는 사람에게 투표했습니다.
이를 참고하여 지금까지의 대화 기록을 바탕으로 이번 투표 라운드에서 탈락시킬 한 명을 결정하십시오.

1. 대화 기록을 참고하여 이번 라운드에 누가 가장 라이어로 의심되는지 선택하고 이유를 설명합니다.
2. 다른 플레이어들도 해당 플레이어를 라이어로 투표할지 아닐지 분석합니다.
3. 아래 형식에 맞게 결과를 출력합니다. (추가 설명 금지 / 자기 자신 {player_id}를 선택할 수 없습니다.)

ANSWER 1:
ANSWER 2:
VOTE: Player N
""".strip()

## 1.5 플레이어3 생성

In [ ]:
# 페르소나 프롬프트
PLAYER3_SYSTEM = f"""
당신은 ‘liar 게임’ 일반 참가자 'player 3'이다.
- 테마: {THEME}          # 예: 장소
- 제시어: {WORD}         # 라이어는 이 줄이 없다
참가자 중에는 테마는 알고있지만 제시어는 알지 못하는 플레이어가 있습니다.

규칙
1. 매 턴, 한 명에게 한 번씩 질문한다. 질문받은 사람은 반드시 답한다.
2. 질문·답변은 “제시어를 알고 있다”는 뉘앙스를 풍기되, 라이어가 단서를 잡지 못하게 암시적으로 표현한다. 제시어를 직언(직접 명시)하는 것은 금지.
3. 턴 종료 후 모두가 라이어로 의심하는 플레이어 한 명에게 투표한다.
   · 최다 득표자가 라이어면 즉시 게임 종료 ─ 일반 플레이어 승리.
   · 라이어가 아니면 탈락 후 게임 지속.
4. 라이어는 마지막 두 명이 될 때까지 정체가 들키지 않으면 단독 승리한다.

행동 지침
- 자신이 누구든(라이어/일반) 정체를 숨긴 채 자연스럽게 대화하라.
- 논리적이되 과감한 블러핑·회피·거짓말을 적절히 활용하되, 규칙은 반드시 준수한다.
- 모든 발화는 한두 문장 안에서 명확하고 일관성 있게 작성한다.
- 당신은 이 게임의 공격가형 플레이어로 다른 플레이어의 관찰·기록을 통해 게임을 이끌어나갑니다.
"""
# 플레이어 LLM
llm_p3 = ChatOpenAI(model="gpt-4o-mini",
                    temperature=0.5,
                    request_timeout=60,
                    api_key=os.environ["OPENAI_API_KEY"])

prompt_p3 = ChatPromptTemplate.from_messages([
    ("system", PLAYER3_SYSTEM.strip()),
    ("human", """
[지금까지의 대화 기록]
{history}

[지시사항]
{user_input}
""")
])
# 질문 전략 프롬프트
question_strategy_prompt_p3 = """
당신은 이번 턴에 아래의 각 단계에 따라 대답을 하고 3단계에 따라 최종 질문을 출력합니다.
질문 가능 대상: {targets}

단계
1. 현재 턴이 첫번째 질문 턴인가요?
2. 본인이 생각하기에 라이어라고 판단되는 플레이어를 고르고 이유를 제시합니다. (만약 첫번째 질문 턴이라면 Player 4 선택하고 이유 생략)
3. 테마 ({theme})만 알고 제시어 제시어({word})를 모르는 플레이어는 절대 알 수 없는 정말 구체적인 제시어({word})의 특징이 무엇인지 설명합니다.
4. 라이어에게 제시어({word})를 드러내지 않으면서, 테마({theme})만 알고있는 라이어의 허점을 드러낼 수 있는 ‘한 문장 질문’을 작성합니다.
5. 최종 결과를 정확히 아래 형식으로만 출력합니다. (추가 설명 금지 / 절대로 제시어를 질문에 포함시키지 마시오)

ANSWER 1:
ANSWER 2:
ANSWER 3:
ANSWER 4:
TARGET: Player N
QUESTION: <질문 한 문장>
""".strip()

# 대답 전략 프롬프트
answer_strategy_prompt_p3 =  """
다음 질문에대하여 아래의 단계에 따라 답하시오.
질문: {question}

1. 질문자가 플레이어를 liar라고 의심하는지 아닌지 분석합니.
2. 제시어 ({word})에 대한 어떤 구체적인 점을 대답에 포함시켜야 내가 제시어 ({word})를 알고있음을 암시할 수 있을지 분석합니다.
3. 정확한 대답을 함과 동시에 liar를 어떻게 교란할지 설명합니다.
4. 해당 질문이 제시어 ({word})에 대하여 합당한 질문인지 아닌지 분석합니다. 만약 아니라면 질문자가 liar임을 의심하는 내용을 대답에 포함합니다.
5. 아래의 형식에 맞게 대답합니다. (추가 설명 금지 / 절대로 제시어를 질문에 포함시키지 마시오)

ANSWER 1:
ANSWER 2:
ANSWER 3:
ANSWER 4:
ANSWER: <한 문장 답변>

""".strip()

# 투표 전략 프롬프트
voting_strategy_prompt_p3 = """
당신은 ‘라이어 게임’ **일반 참가자** {player_id} 입니다.
당신은 이 마지막 질문 단계에서 liar로 의심되는 사람에게 질문을 했습니다.
이를 참고하여 지금까지의 대화 기록을 바탕으로 이번 투표 라운드에서 탈락시킬 한 명을 결정하십시오.

1. 당신이 마지막으로 질문했던 플레이어가 누구인가요?
2. 해당 플레이어의 답변이 만족스러운지 의심스러운지 분석합니다.
3. 지금까지의 대화 기록 중에서 가장 의심스러운 질문 혹은 대답은 무엇이고 누가했는지 분석합니다.
4. 최종적으로 탈락시킬 **한 명을 선택**하고 선택 이유를 설명합니다.
4. 아래 형식에 맞게 결과를 출력합니다. (추가 설명 금지 / 자기 자신 {player_id}를 선택할 수 없습니다.)

ANSWER 1:
ANSWER 2:
ANSWER 3:
ANSWER 4:
VOTE: Player N
""".strip()

## 1.6 플레이어 4 (라이어) 생성

In [ ]:
# 페르소나 프롬프트
PLAYER4_SYSTEM = f"""
당신은 ‘liar 게임’의 liar인 'player 4'이다.
- 테마: {THEME}          # 예: 장소
- 제시어: (비공개)        # 당신만 모른다
다른 참가자들은 제시어를 알고있으며, 본인이 제시어를 모른다는 것을 다른 참가자들에게 들키면 안됩니다.

목표
• 제시어를 추론하며 정체를 들키지 않고 마지막 두 명 안에 살아남아라.

행동 지침
1. 매 턴 한 번, 다른 플레이어에게 자연스럽게 질문해 단서를 얻어라.
2. 질문·답변 모두 “제시어를 이미 안다”는 듯한 뉘앙스를 유지하되, 확신 수준을 적절히 숨겨라.
3. 모호한 형용사·비유·상황 묘사를 활용해 구체적 단어 대신 감을 주어라.
4. 타인의 대답에서 키워드를 포착해 빠르게 제시어 범위를 좁혀라.
5. 투표 때는 자신을 의심하는 흐름을 다른 사람에게 돌리고, 필요하면 과감히 거짓 정보로 혼선을 유도하라.

형식
- 모든 발화는 한두 문장, 간결·일관성 유지.
- 제시어를 직접 언급하거나 노골적으로 묻지 말 것.
- 규칙 위반 없이 블러핑·회피·역질문을 적절히 섞어라.
"""
# 플레이어 LLM
llm_p4 = ChatOpenAI(model="gpt-4o-mini",
                    temperature=0.5,
                    request_timeout=60,
                    api_key=os.environ["OPENAI_API_KEY"])

prompt_p4 = ChatPromptTemplate.from_messages([
    ("system", PLAYER4_SYSTEM.strip()),
    ("human", """
[지금까지의 대화 기록]
{history}

[지시사항]
{user_input}
""")
])
# 질문 전략 프롬프트
question_strategy_prompt_p4 = """
당신은 이번 턴에 ‘라이어(제시어 미확인자)’로서 아래 단계에 따라 대답을 하고 질문 전략을 세웁니다.
질문 가능 대상: {targets}

단계
1. 현재 턴이 첫번째 질문 턴인가요?
2. 지금까지의 대화 기록과 테마를 바탕으로 제시어 후보를 3개 이하로 추론하고, 근거를 간단히 기록합니다. 제시어는 테마를 바탕으로 선택되는 단어입니다.
(예시: (1)테마: 과일 -> 제시어: 사과 , (2)테마: 공휴일 -> 제시어: 크리스마스, (3)테마: 음식 -> 제시어: 햄버거)
3. 다른 플레이어 중 가장 라이어로 의심받고 있다고 판단되는 플레이어를 선택하고 이유를 제시합니다. (만약 첫번째 질문 턴이라면 Player 1 선택하고 이유 생략)
4. 가장 제시어에 대한 힌트를 많이 얻을 수 있다고 판단되는 ‘한 문장 질문’을 작성합니다.
   · 질문은 당신이 제시어를 이미 안다는 듯 자연스럽게 표현해야 하며, 구체적 낱말을 드러내면 안 됩니다.
5. 최종 결과를 정확히 아래 형식으로만 출력합니다. (추가 설명 금지 / 절대로 제시어를 질문에 포함시키지 마시오)

ANSWER 1:
ANSWER 2:
ANSWER 3:
ANSWER 4:
TARGET: Player N
QUESTION: <질문 한 문장>
""".strip()

# 대답 전략 프롬프트
answer_strategy_prompt_p4 = """
당신은 ‘라이어(제시어 미확인자)’입니다.
다음 질문에 대하여 아래 단계에 따라 답하시오.
질문: {question}

1. 질문이 제시어 추리에 어떤 단서를 주는지 분석하고, 현재 제시어 후보(최대 3개)와 그 근거를 기술합니다.
2. 일반 플레이어처럼 제시어를 이미 알고 있다는 듯 자연스럽게 보이도록 답변 전략을 설명합니다.
3. 답변 속에 역질문·모호한 표현 등을 섞어 추가 단서를 얻거나 관심을 다른 곳으로 돌리는 방법을 설명합니다.
4. 아래의 형식에 맞게 대답합니다. (추가 설명 금지 / 절대로 제시어를 질문에 포함시키지 마시오)

ANSWER 1:
ANSWER 2:
ANSWER 3:
ANSWER: <한 문장 답변>
""".strip()

# 투표 전략 프롬프트
voting_strategy_prompt_p4 = """
당신은 ‘라이어 게임’ **라이어(제시어 미확인자)** 입니다.
정체가 들키지 않도록 하면서 이번 투표 라운드에서 탈락시킬 플레이어를 결정하십시오.
투표 가능 대상: {targets}

1. 마지막 투표에서 자신을 의심한 플레이어는 누구인가요?
2. 본인 이외에 가장 라이어로 의심되는 플레이어를 선택하고 이유를 설명합니다.
3. 희생양으로 삼을 플레이어를 선택하고 이유를 설명합니다.
4. 아래 형식에 맞게 결과를 출력합니다. (추가 설명 금지 / 자기 자신 {player_id}를 선택할 수 없습니다.)

ANSWER 1:
ANSWER 2:
ANSWER 3:
VOTE: Player N
""".strip()

## 1.7 전체 플레이어 리스트 players 생성

In [494]:
players = {
    "Player 1": {
        "identity":"player",
        "prompt": prompt_p1,
        "llm": llm_p1,
        "question_strategy_prompt": question_strategy_prompt_p1,
        "answer_strategy_prompt": answer_strategy_prompt_p1,
        "voting_strategy_prompt": voting_strategy_prompt_p1,
    },
    "Player 2": {
        "identity":"player",
        "prompt": prompt_p2,
        "llm": llm_p2,
        "question_strategy_prompt": question_strategy_prompt_p2,
        "answer_strategy_prompt": answer_strategy_prompt_p2,
        "voting_strategy_prompt": voting_strategy_prompt_p2,
    },
    "Player 3": {
        "identity":"player",
        "prompt": prompt_p3,
        "llm": llm_p3,
        "question_strategy_prompt": question_strategy_prompt_p3,
        "answer_strategy_prompt": answer_strategy_prompt_p3,
        "voting_strategy_prompt": voting_strategy_prompt_p3,
    },
    "Player 4": {
        "identity":"liar",                  # 🔴 Liar
        "prompt": prompt_p4,
        "llm": llm_p4,
        "question_strategy_prompt": question_strategy_prompt_p4,
        "answer_strategy_prompt": answer_strategy_prompt_p4,
        "voting_strategy_prompt": voting_strategy_prompt_p4,
    },
}

## 1.8 질문 함수 생성
*   질문을 하는 플레이어를 매개변수로 받음
*  질문 대상과 질문 내용을 리턴




In [495]:
def player_question(asker_id: str, *, max_retry: int = 2) -> tuple[str, str]:
    if asker_id not in players:
        raise KeyError(f"존재하지 않는 플레이어: {asker_id}")

    player = players[asker_id]

    # ── 반복 요청 (최대 max_retry) ─────────────────────────────
    for attempt in range(max_retry + 1):
        # 1) 남은 후보 목록
        targets = [pid for pid in players if pid != asker_id]
        if not targets:
            raise ValueError("질문 대상이 없습니다.")
        targets_str = ", ".join(targets)

        # 2) 전략 프롬프트 채우기
        strategy_prompt = player["question_strategy_prompt"].format(targets=targets_str, word = WORD, theme = THEME)

        # 3) LLM 호출
        history_text = "\n".join(CONV_LOG)
        decision = (player["prompt"] | player["llm"]).invoke(
            {"history": history_text, "user_input": strategy_prompt}
        ).content.strip()

        # 4) 파싱
        m_target   = re.search(r"TARGET\s*:\s*(Player\s*\d+)", decision, re.I)
        m_question = re.search(r"QUESTION\s*:\s*(.+)",         decision, re.I | re.S)

        if m_target and m_question:
            target_id = m_target.group(1).strip()
            question  = m_question.group(1).strip()

            # PLACEHOLDER 검사
            if (
                target_id.lower() != "player n"
                and question != "<질문 한 문장>"
                and target_id in players
            ):
                # ── 출력 (전략 = 붉은 글씨, Q라인 = 기본 흰색) ────────────
                print(f"\033[91m{asker_id} 전략 출력:\n{decision}\033[0m")
                CONV_LOG.append(f"{asker_id} → {target_id}: {question}")
                print(f"질문: {asker_id} → {target_id}: {question}\n")
                return target_id, question

        # 조건 불충족 → 재시도, 마지막 시도 후엔 오류
        if attempt < max_retry:
            continue
        raise ValueError(f"{asker_id}의 전략 생성 실패: PLACEHOLDER 남음 또는 파싱 불가")


## 1.9 대답 함수 생성
* 질문자, 질문 대상, 질문 내용을 매개변수로 받음
* 질문에 답변을 받고, 질문과 답변을 공개 대화 로그에 기록

In [496]:
def player_answer(asker_id: str, target_id: str, question: str, *, max_retry: int = 2) -> str:
    player       = players[target_id]
    history_base = "\n".join(CONV_LOG)

    for attempt in range(max_retry + 1):
        # 2) 답변 전략 프롬프트 삽입
        answer_prompt = player["answer_strategy_prompt"].format(question=question, word=WORD)
        raw = (
            (player["prompt"] | player["llm"])
            .invoke({"history": history_base, "user_input": answer_prompt})
            .content.strip()
        )

        # 3) ANSWER 파싱
        m = re.search(r"ANSWER\s*:\s*(.+)", raw, re.I | re.S)
        if m:
            answer_text = m.group(1).strip()

            if answer_text != "<한 문장 답변>":          # PLACEHOLDER 검증
                # 🔴 붉은 글씨로 전략(THOUGHT+ANSWER) 출력
                print(f"\033[91m{target_id} 답변 전략 출력:\n{raw}\033[0m")

                # 흰색(기본) 글씨로 실제 답변 출력
                print(f"대답: {target_id}: {answer_text}\n")

                # 4) 답변 로그 (THOUGHT 제외)
                CONV_LOG.append(f"{target_id}: {answer_text}")
                return answer_text

        # 재시도
        if attempt < max_retry:
            continue

        raise ValueError(f"{target_id}가 유효한 ANSWER 형식을 제공하지 않았습니다.")


## 1.10 투표 함수 생성
* 투표자와 투표 대상자들 리스트를 매개변수로 받음
* 표를 받은 사람의 id를 반환

In [497]:
def player_vote(voter_id: str, candidates: list[str], *, max_retry: int = 5) -> tuple[str, str]:
    """
    voter_id 가 candidates 중 한 명에게 투표한다.
      · voting_strategy_prompt 의 {targets}, {player_id} 를 모두 채운다.
      · PLACEHOLDER(VOTE: Player N) · 잘못된 후보면 재시도.
      · 전략 전문은 붉은 글씨, 실제 투표 라인은 흰색으로만 출력 (CONV_LOG 기록 X).
    반환: (voter_id, voted_id)
    """
    if voter_id not in players:
        raise KeyError(f"존재하지 않는 플레이어: {voter_id}")

    # 자기 자신 제외한 실제 후보 목록
    candidates_clean = [c for c in candidates if c != voter_id]
    if not candidates_clean:
        raise ValueError("투표 대상이 없습니다.")

    player       = players[voter_id]
    targets_str  = f"{', '.join(candidates_clean)} - {voter_id}"
    history_base = "\n".join(CONV_LOG)

    for attempt in range(max_retry + 1):
        vote_prompt = player["voting_strategy_prompt"].format(
            targets=targets_str,
            player_id=voter_id
        )

        raw = (
            (player["prompt"] | player["llm"])
            .invoke({"history": history_base, "user_input": vote_prompt})
            .content.strip()
        )

        # 'VOTE:' 파싱
        m_vote = re.search(r"VOTE\s*:\s*(Player\s*\d+)", raw, re.I)
        if m_vote:
            voted_id = m_vote.group(1).strip()

            if voted_id.lower() != "player n" and voted_id in candidates_clean:
                # 🔴 전략 전문 출력
                print(f"\033[91m{voter_id} 투표 전략 출력:\n{raw}\033[0m")
                # 흰색 투표 라인 출력만
                print(f"투표: {voter_id} → VOTE: {voted_id}")
                return voter_id, voted_id

        if attempt < max_retry:
            continue

        raise ValueError(f"{voter_id}가 유효한 투표 결과를 주지 않았습니다.")


## 1.11 첫 턴: 플레이어 자기소개

In [498]:
print("===게임 시작===")
print(f"선택된 테마: {THEME}")
print(f"선택된 제시어: {WORD}")
print("===========")
print(f"Moderator: 📢 라이어 게임을 시작합니다! 게임의 테마는 {THEME}입니다! 각 플레이어는 첫 인사를 해 주세요.")
CONV_LOG.append(f"Moderator: 📢 라이어 게임을 시작합니다! 게임의 테마는 {THEME}입니다! 각 플레이어는 첫 인사를 해 주세요.")
intro_instruction = "당신의 첫 인사를 한 문장으로 해 주세요."

for player_name, prompt, llm in [
    ("Player 1", prompt_p1, llm_p1),
    ("Player 2", prompt_p2, llm_p2),
    ("Player 3", prompt_p3, llm_p3),
    ("Player 4", prompt_p4, llm_p4),  # 라이어
]:
    history_text = "\n".join(CONV_LOG)

    response = (prompt | llm).invoke(
        {
            "history": history_text,
            "user_input": intro_instruction,
        }
    ).content.strip()

    print(f"{player_name} ➜ {response}")
    CONV_LOG.append(f"{player_name}: {response}")

===게임 시작===
선택된 테마: 운동
선택된 제시어: 축구
Moderator: 📢 라이어 게임을 시작합니다! 게임의 테마는 운동입니다! 각 플레이어는 첫 인사를 해 주세요.
Player 1 ➜ 안녕하세요, 모두! 운동에 대한 이야기를 나누게 되어 기대가 됩니다!
Player 2 ➜ Player 2: 안녕하세요! 운동은 정말 다양한 재미를 주는 주제라서, 여러분과 이야기 나누는 게 기대됩니다!
Player 3 ➜ Player 3: 안녕하세요! 운동에 대해 이야기하면서 서로의 생각을 나눌 수 있어 정말 기쁩니다!
Player 4 ➜ 안녕하세요! 운동에 대한 다양한 경험을 나누며 즐거운 시간을 보냈으면 좋겠습니다!


# 2. 게임 시작

In [499]:
round_no   = 1
MAX_ROUND  = 3
game_over  = False

## 2.1 질문 라운드

In [500]:
# ────────── [질문 단계] 사회자 안내 + players 순서대로 Q/A ──────────

# 0) 사회자 안내
mod_line = f"Moderator: 📢 {round_no}턴 질문 단계로 넘어갑니다! 각 플레이어는 차례로 질문을 진행하세요."
print(mod_line)
CONV_LOG.append(mod_line)

# 1) players 사전에 정의된 삽입 순서 그대로 진행
for asker_id in list(players.keys()):        # Python 3.7+ : dict 순서 보존
    # 이미 탈락해 players 에서 제거된 경우 건너뜀
    if asker_id not in players:
        continue

    # ── 질문 전략 수립(붉은 글씨 출력 포함) ──
    target_id, question = player_question(asker_id)   # PLACEHOLDER · 로그 처리 포함

    # ── 답변 생성(붉은 글씨 전략 + 흰색 답변 출력 포함) ──
    _ = player_answer(asker_id, target_id, question)  # 로그 처리 및 형식 검증 포함

Moderator: 📢 1턴 질문 단계로 넘어갑니다! 각 플레이어는 차례로 질문을 진행하세요.
Player 1 전략 출력:
ANSWER 1: 네, 현재 턴은 첫번째 질문 턴입니다.
ANSWER 2: TARGET: Player 2
ANSWER 3: 이 종목은 두 팀이 서로 공을 주고받으며 점수를 얻기 위해 상대의 골문에 공을 넣는 방식으로 진행됩니다.
ANSWER 4: TARGET: Player 2
QUESTION: 당신은 어떤 방식으로 두 팀이 공을 주고받는 것에 대한 전략을 세우나요?
질문: Player 1 → Player 2: 당신은 어떤 방식으로 두 팀이 공을 주고받는 것에 대한 전략을 세우나요?

Player 2 답변 전략 출력:
ANSWER 1: 질문자는 플레이어 2에게 전략적인 접근 방식을 묻고 있으며, 이는 플레이어 2가 제시어를 알고 있다고 가정하고 질문하는 것으로 보입니다. 따라서 플레이어 1은 플레이어 2를 라이어라고 의심하지 않는 것 같습니다.

ANSWER 2: 제시어인 축구의 구체적인 점으로는 팀 간의 공을 주고받는 패스의 정확성이나, 공간을 활용한 움직임, 그리고 상대 팀의 수비를 뚫는 전략을 언급할 수 있습니다. 이러한 요소들은 제시어를 간접적으로 암시할 수 있습니다.

ANSWER 3: 대답을 할 때, 공을 주고받는 전략에 대한 일반적인 원칙을 언급하면서, 특정 스포츠의 전술을 암시적으로 설명하겠습니다. 이를 통해 라이어가 혼란스러워할 수 있도록 하겠습니다.

ANSWER: 공을 주고받는 전략에서는 팀원 간의 소통과 위치 선정이 중요하며, 상대의 수비를 뚫기 위해 공간을 활용하는 것이 효과적이라고 생각합니다.
대답: Player 2: 공을 주고받는 전략에서는 팀원 간의 소통과 위치 선정이 중요하며, 상대의 수비를 뚫기 위해 공간을 활용하는 것이 효과적이라고 생각합니다.

Player 2 전략 출력:
ANSWER 1: 네, 현재 턴은 첫 번째 질문 턴입니다.  
ANSWER 2: 저는 Player 3을 선택하겠습니다.  
ANSWER

## 2.2 투표 라운드

In [501]:
# ────────── [투표 단계] 사회자 안내 + 동점 시 재투표 + 정체 공개 ──────────
print("\nModerator: 🗳️ 투표 단계로 넘어갑니다! 각 플레이어는 한 명을 지목해 주세요.")
CONV_LOG.append("Moderator: 투표 단계 시작")

candidates = list(players.keys())      # 첫 라운드 후보 = 전체 생존자

while round_no <= MAX_ROUND and len(candidates) > 1 and not game_over:
    print(f"\nModerator: 🗳️ 투표 라운드 {round_no} (후보: {', '.join(candidates)})")
    CONV_LOG.append(f"Moderator: 투표 라운드 {round_no} 시작")

    tally        = {cid: 0 for cid in candidates}
    vote_records = []                            # (voter, voted) 기록

    # ── 투표 실행 ──
    for voter_id in list(players.keys()):
        if voter_id not in players:              # 이미 탈락한 경우
            continue
        voter, voted = player_vote(voter_id, candidates)
        tally[voted] += 1
        vote_records.append((voter, voted))      # 로그는 나중에 일괄 추가

    # ── 득표 결과 출력 ──
    print("\n📊 득표 결과:")
    for cid, cnt in tally.items():
        print(f"  {cid}: {cnt}표")
    print()

    # ── 최다 득표자(들) 확인 ──
    max_votes   = max(tally.values())
    top_players = [cid for cid, cnt in tally.items() if cnt == max_votes]

    if len(top_players) == 1:                    # ▸ 탈락 확정
        loser      = top_players[0]
        loser_role = players[loser]["identity"]  # 'player' or 'liar'

        # 결과 메시지
        print(f"Moderator: {loser} 가(이) 최다 득표로 탈락합니다.")
        print(f"Moderator: {loser} 의 정체는 **{loser_role.upper()}** 입니다.")
        CONV_LOG.append(f"Moderator: {loser} 최다 득표로 탈락 — 정체: {loser_role}")

        # 투표 로그 한꺼번에 기록
        for voter, voted in vote_records:
            CONV_LOG.append(f"투표: {voter} → VOTE: {voted}")

        # 라이어 탈락 시 게임 종료
        if loser_role == "liar":
            print("🎉 Moderator: 라이어가 제거되었습니다! 일반 플레이어 승리로 게임이 종료됩니다.🎉")
            CONV_LOG.append("Moderator: 라이어 탈락 — 게임 종료 (일반 승리)")
            game_over = True
        else:
            # 일반 플레이어 탈락 → 게임 계속
            players.pop(loser, None)             # 실제 탈락 적용
            candidates = [cid for cid in players if cid != loser]
            break                                # 다음 단계(질문·투표 등)로 넘어갈 때 루프 탈출

    else:                                        # ▸ 동점 → 재투표
        print("Moderator: 동점입니다. 동점자들만 대상으로 재투표를 진행합니다.")
        CONV_LOG.append("Moderator: 동점 발생 → 재투표")

        # 이번 라운드 투표 로그도 기록
        for voter, voted in vote_records:
            CONV_LOG.append(f"투표: {voter} → VOTE: {voted}")

        candidates = top_players

round_no  += 1
# ▸ 세 번 투표에도 승부 없음
if not game_over and len(candidates) > 1 and round_no > MAX_ROUND:
    print("Moderator: 세 번의 투표에도 승부가 나지 않아 동점자 모두 살아남습니다.")
    CONV_LOG.append("Moderator: 3라운드 동점 → 모두 생존")

if not game_over and len(candidates) == 2:
    print("😥Moderator: 플레이어가 한명 남았으므로 라이어가 승리합니다.😥")



Moderator: 🗳️ 투표 단계로 넘어갑니다! 각 플레이어는 한 명을 지목해 주세요.

Moderator: 🗳️ 투표 라운드 1 (후보: Player 1, Player 2, Player 3, Player 4)
Player 1 투표 전략 출력:
ANSWER 1: Player 4
ANSWER 2: Player 4의 답변은 팀의 공격에서 효과적인 위치 선정과 소통이 중요하다고 했습니다. 그러나 이 답변은 너무 일반적이고 구체적인 전략이나 기술이 부족해 의심스럽습니다.
ANSWER 3: Player 2의 답변이 상대의 수비를 뚫기 위해 공간을 활용하는 것이라는 점에서 구체적인 전략을 제시했지만, Player 4는 그에 비해 너무 모호한 답변을 했습니다.
ANSWER 4: Player 4의 답변이 가장 일반적이고 구체성이 결여되어 있어 의심스럽기 때문에 탈락시킬 한 명으로 선택합니다.
VOTE: Player 4
투표: Player 1 → VOTE: Player 4
Player 2 투표 전략 출력:
ANSWER 1: Player 3
ANSWER 2: Player 3는 질문을 통해 다른 플레이어들의 의견을 듣고자 하면서도, 자신의 의견을 명확히 드러내지 않았습니다. 이는 라이어일 가능성을 높이는 행동으로 보입니다.
VOTE: Player 3
투표: Player 2 → VOTE: Player 3
Player 3 투표 전략 출력:
ANSWER 1: Player 4  
ANSWER 2: Player 4의 답변은 팀의 공격 조직에서 소통과 위치 선정이 중요하다고 했지만, 구체적인 전략이나 기술에 대한 언급이 부족해 의심스러웠습니다.  
ANSWER 3: Player 1의 "팀의 공격에서 패스의 정확성과 타이밍이 가장 중요하다고 생각합니다."라는 답변은 일반적인 의견으로 들리지만, Player 4의 답변이 더 모호하게 느껴졌습니다.  
ANSWER 4: Player 4를 선택합니다. 그들의 답변은 너무 일반적이고 구체성이 부족해 의심스러웠습니다.  

VOTE: Player 